In [0]:
from pyspark.sql.functions import col, current_timestamp, when

In [0]:
latest_vitals_df = spark.table(
    "healthcare_analytics.gold_patient_latest_vitals"
)

alerts_df = spark.table(
    "healthcare_analytics.gold_patient_alerts"
)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

alert_window = Window.partitionBy("patient_id").orderBy(col("alert_generated_ts").desc())

latest_alerts_df = (
    alerts_df
    .withColumn("rn", row_number().over(alert_window))
    .filter(col("rn") == 1)
    .drop("rn")
)

In [0]:
snapshot_df = (
    latest_vitals_df.alias("v")
    .join(
        latest_alerts_df.alias("a"),
        col("v.patient_id") == col("a.patient_id"),
        "left"
    )
    .withColumn(
        "risk_group",
        when(col("a.overall_severity") == "HIGH", "CRITICAL")
        .when(col("a.overall_severity") == "MEDIUM", "MODERATE")
        .otherwise("STABLE")
    )
    .select(
        col("v.patient_id"),
        col("v.heart_rate"),
        col("v.oxygen_level"),
        col("v.systolic_bp"),
        col("a.overall_severity"),
        col("risk_group"),
        col("a.alert_generated_ts").alias("last_alert_ts"),
        current_timestamp().alias("snapshot_ts"),
        col("v.source_system")
    )
)

In [0]:
(
    snapshot_df.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("healthcare_analytics.gold_patient_risk_snapshot")
)

In [0]:
spark.table(
    "healthcare_analytics.gold_patient_risk_snapshot"
).display()

patient_id,heart_rate,oxygen_level,systolic_bp,overall_severity,risk_group,last_alert_ts,snapshot_ts,source_system
496,91,90,90,NORMAL,STABLE,2026-01-18T22:42:37.735Z,2026-01-19T05:07:17.869Z,simulated_vitals
148,94,100,102,NORMAL,STABLE,2026-01-18T22:42:37.735Z,2026-01-19T05:07:17.869Z,simulated_vitals
463,96,87,106,CRITICAL,STABLE,2026-01-18T22:42:37.735Z,2026-01-19T05:07:17.869Z,simulated_vitals
392,82,93,104,NORMAL,STABLE,2026-01-18T22:42:37.735Z,2026-01-19T05:07:17.869Z,simulated_vitals
243,113,90,103,HIGH,CRITICAL,2026-01-18T22:42:37.735Z,2026-01-19T05:07:17.869Z,simulated_vitals
31,124,98,102,CRITICAL,STABLE,2026-01-18T22:42:37.735Z,2026-01-19T05:07:17.869Z,simulated_vitals
451,65,95,104,NORMAL,STABLE,2026-01-18T22:42:37.735Z,2026-01-19T05:07:17.869Z,simulated_vitals
251,95,99,172,CRITICAL,STABLE,2026-01-18T22:42:37.735Z,2026-01-19T05:07:17.869Z,simulated_vitals
458,126,93,126,CRITICAL,STABLE,2026-01-18T22:42:37.735Z,2026-01-19T05:07:17.869Z,simulated_vitals
65,106,91,169,CRITICAL,STABLE,2026-01-18T22:42:37.735Z,2026-01-19T05:07:17.869Z,simulated_vitals
